# OMERO Micro-SAM Annotation Workflow

This notebook demonstrates the new streamlined workflow using the `omero-annotate-ai` package focused on micro-SAM.

### Features:
- Interactive configuration widget
- Support for all micro-SAM models (vit_b, vit_l, vit_h, vit_b_lm)
- YAML-based configuration
- Clean API with backward compatibility
- 2D and 3D processing modes

### Instructions:
- Login: To run with OMERO and to not expose login in the notebook, the username is stored in .env file (see example .env_example). The password needs to be typed every time.
- This notebook supports processing images from various OMERO object types: images, datasets, projects, plates, and screens.
- Specify the container type and ID using the interactive widget below.

## 1. Install and Import the Package

**Note**: If you haven't installed the package yet, run:
```bash
pip install -e .
```

For OMERO functionality, also install:
```bash
pip install -e .[omero]
```

In [1]:
# Import the new package
import omero_annotate_ai
from omero_annotate_ai import create_config_widget, create_pipeline, create_default_config

# OMERO-related imports
import omero
from omero.gateway import BlitzGateway
try:
    import ezomero
    OMERO_AVAILABLE = True
except ImportError:
    print("⚠️  ezomero not available. Install with: pip install -e .[omero]")
    OMERO_AVAILABLE = False

# System and utility imports
import os
import tempfile
import shutil
from dotenv import load_dotenv
from getpass import getpass

print(f"📦 omero-annotate-ai version: {omero_annotate_ai.__version__}")
print(f"🔗 OMERO functionality: {'✅ Available' if OMERO_AVAILABLE else '❌ Not available'}")

📦 omero-annotate-ai version: 0.1.0
🔗 OMERO functionality: ✅ Available


## 2. Setup Connection with OMERO

In [2]:
if not OMERO_AVAILABLE:
    raise ImportError("OMERO functionality not available. Please install ezomero.")

# Load environment variables
load_dotenv(override=True)

# Ask for password if not set
if not os.environ.get("PASSWORD"):
    os.environ["PASSWORD"] = getpass("Enter OMERO server password: ")

# Connect to OMERO
conn = BlitzGateway(
    host=os.environ.get("HOST"), 
    username=os.environ.get("USER_NAME"), 
    passwd=os.environ.get("PASSWORD"), 
    group=os.environ.get("GROUP"), 
    secure=True
)

connection_status = conn.connect()
if connection_status:
    print("✅ Connected to OMERO Server")
    print(f"👤 User: {conn.getUser().getName()}")
    print(f"🏢 Group: {conn.getGroupFromContext().getName()}")
else:
    print("❌ Connection to OMERO Server Failed")
    raise ConnectionError("Could not connect to OMERO")

conn.c.enableKeepAlive(60)

# Create temporary work directory
output_directory = os.path.normcase(tempfile.mkdtemp())
print(f"📁 Created temporary work directory: {output_directory}")

✅ Connected to OMERO Server
👤 User: root
🏢 Group: system
📁 Created temporary work directory: c:\users\maarten\appdata\local\temp\tmp9ah_n5te


## 3. Interactive Configuration

Use the interactive widget to configure your micro-SAM annotation workflow. The widget provides:
- **OMERO Connection**: Specify container type and ID
- **Micro-SAM Model**: Choose model type (vit_b, vit_l, vit_h, vit_b_lm)
- **Image Processing**: Configure timepoints, z-slices, channels, 3D mode
- **Patches**: Optional patch extraction settings
- **Training**: Data organization and validation
- **Workflow**: Resume options and output settings

In [3]:
# Create the configuration widget
config_widget = create_config_widget()
config_widget.display()

print("📝 Configure your micro-SAM annotation workflow using the widget above:")
print("   1. Set your OMERO container details (Dataset ID, etc.)")
print("   2. Choose your micro-SAM model (vit_b, vit_l, vit_h, vit_b_lm)")
print("   3. Adjust processing parameters as needed")
print("   4. Click 'Update Configuration' to apply changes")
print("   5. Click 'Validate' to check your configuration")
print("   6. Click 'Show YAML' to see the configuration")

📝 Configure your micro-SAM annotation workflow using the widget above:
   1. Set your OMERO container details (Dataset ID, etc.)
   2. Choose your micro-SAM model (vit_b, vit_l, vit_h, vit_b_lm)
   3. Adjust processing parameters as needed
   4. Click 'Update Configuration' to apply changes
   5. Click 'Validate' to check your configuration
   6. Click 'Show YAML' to see the configuration


## 4. Optional: Set Training Set Name

Choose a specific name for your training set. This helps with organization and allows resuming interrupted workflows.

In [6]:
import pandas as pd

# Set a name for the training set
# Use a specific name if you want to resume from an existing table
trainingset_name = "training_data_20240607"  # Fixed name for resuming
# trainingset_name = "training_data_" + pd.Timestamp.now().strftime("%Y%m%d_%H%M")  # Auto-generated name

print(f"🎯 Training Set Name: {trainingset_name}")

# Update the configuration with the training set name
config = config_widget.get_config()
config.training.trainingset_name = trainingset_name
config.batch_processing.output_folder = output_directory

print(f"📁 Output directory: {output_directory}")

🎯 Training Set Name: training_data_20240607
📁 Output directory: c:\users\maarten\appdata\local\temp\tmp9ah_n5te


## 5. Validate Configuration and Show Preview

Before running the pipeline, let's validate the configuration and preview what will be processed.

In [7]:
# Update configuration from widget
config = config_widget.get_config()
config.training.trainingset_name = trainingset_name
config.batch_processing.output_folder = output_directory

# Validate configuration
try:
    config.validate()
    print("✅ Configuration is valid!")
except ValueError as e:
    print(f"❌ Configuration validation failed:")
    print(f"   {e}")
    raise

# Show configuration summary
print("\n📋 Configuration Summary:")
print(f"   🔬 Model: {config.microsam.model_type}")
print(f"   📦 Container: {config.omero.container_type} (ID: {config.omero.container_id})")
print(f"   📺 Channel: {config.omero.channel}")
print(f"   🎯 Training Set: {config.training.trainingset_name}")
print(f"   📊 Batch Size: {config.batch_processing.batch_size}")
print(f"   🧩 Use Patches: {config.patches.use_patches}")
if config.patches.use_patches:
    print(f"   📐 Patch Size: {config.patches.patch_size}")
    print(f"   📈 Patches per Image: {config.patches.patches_per_image}")
print(f"   🔄 Resume from Table: {config.workflow.resume_from_table}")
print(f"   📖 Read-only Mode: {config.workflow.read_only_mode}")
print(f"   🧠 3D Processing: {config.microsam.three_d}")

✅ Configuration is valid!

📋 Configuration Summary:
   🔬 Model: vit_l
   📦 Container: project (ID: 101)
   📺 Channel: 0
   🎯 Training Set: training_data_20240607
   📊 Batch Size: 3
   🧩 Use Patches: False
   🔄 Resume from Table: False
   📖 Read-only Mode: False
   🧠 3D Processing: False


## 6. Preview Container Contents

Let's check what images will be processed from your selected container.

In [8]:
# Create the pipeline (but don't run it yet)
pipeline = create_pipeline(config, conn)

# Get container details
container_type = config.omero.container_type
container_id = config.omero.container_id

# Validate container exists and get details
if container_type == "dataset":
    container = conn.getObject("Dataset", container_id)
    if container is None:
        raise ValueError(f"Dataset with ID {container_id} not found")
    print(f"📁 Dataset: {container.getName()} (ID: {container_id})")
    print(f"📝 Description: {container.getDescription() or 'No description'}")
    
elif container_type == "project":
    container = conn.getObject("Project", container_id)
    if container is None:
        raise ValueError(f"Project with ID {container_id} not found")
    print(f"📂 Project: {container.getName()} (ID: {container_id})")
    print(f"📝 Description: {container.getDescription() or 'No description'}")
    
elif container_type == "plate":
    container = conn.getObject("Plate", container_id)
    if container is None:
        raise ValueError(f"Plate with ID {container_id} not found")
    print(f"🧪 Plate: {container.getName()} (ID: {container_id})")
    print(f"📝 Description: {container.getDescription() or 'No description'}")
    
elif container_type == "image":
    container = conn.getObject("Image", container_id)
    if container is None:
        raise ValueError(f"Image with ID {container_id} not found")
    print(f"🖼️ Image: {container.getName()} (ID: {container_id})")
    
else:
    raise ValueError(f"Unsupported container type: {container_type}")

# Get list of images that will be processed
try:
    images_list = pipeline.get_images_from_container()
    print(f"\n📊 Found {len(images_list)} images to process")
    
    # Show first few images
    print("\n🖼️ Sample images:")
    for i, img in enumerate(images_list[:5]):
        if hasattr(img, 'getName'):  # OMERO image object
            print(f"   {i+1}. {img.getName()} (ID: {img.getId()})")
        else:  # Image ID
            img_obj = conn.getObject("Image", img)
            if img_obj:
                print(f"   {i+1}. {img_obj.getName()} (ID: {img})")
    
    if len(images_list) > 5:
        print(f"   ... and {len(images_list) - 5} more images")
        
except Exception as e:
    print(f"❌ Error getting images from container: {e}")
    raise

📂 Project: Senescence (ID: 101)
📝 Description: No description
📁 Loading images from project 101
📊 Found 6 images

📊 Found 6 images to process

🖼️ Sample images:
   1. r01c06.tif (ID: 252)
   2. r01c05.tif (ID: 255)
   3. r01c02.tif (ID: 251)
   4. r01c01.tif (ID: 256)
   5. r01c04.tif (ID: 253)
   ... and 1 more images


## 7. Run the Annotation Pipeline

Now we'll run the annotation workflow. This will:
1. Create or resume from a tracking table
2. Process images in batches
3. Launch napari for interactive annotation
4. Upload results to OMERO (or save locally if in read-only mode)

In [9]:
print("🚀 Starting annotation pipeline...")
print(f"   This will process {len(images_list)} images using micro-SAM")
print(f"   Model: {config.microsam.model_type}")
print(f"   Napari will open for interactive annotation")
print(f"   Close napari windows when annotation is complete for each batch")

try:
    # Run the complete workflow
    table_id, processed_images = pipeline.run_full_workflow()
    
    print(f"\n✅ Annotation pipeline completed successfully!")
    print(f"📊 Processed {len(processed_images)} images")
    print(f"📋 Tracking table ID: {table_id}")
    
    if config.workflow.read_only_mode:
        print(f"💾 Annotations saved locally to: {config.workflow.local_output_dir}")
    else:
        print(f"☁️ Annotations uploaded to OMERO")
        
except Exception as e:
    print(f"❌ Error during annotation pipeline: {e}")
    raise

🚀 Starting annotation pipeline...
   This will process 6 images using micro-SAM
   Model: vit_l
   Napari will open for interactive annotation
   Close napari windows when annotation is complete for each batch
📁 Loading images from project 101
📊 Found 6 images
🚀 Starting micro-SAM annotation pipeline
📊 Processing 6 images with model: vit_l
📋 Creating new tracking table: microsam_training_training_data_20240607
📋 Would create tracking table 'microsam_training_training_data_20240607' with 6 units
   Container: project 101


{
    _id = <nil>
    _details = object #1 (::omero::model::Details)
    {
        _owner = <nil>
        _group = <nil>
        _creationEvent = <nil>
        _updateEvent = <nil>
        _permissions = <nil>
        _externalInfo = <nil>
        _call = {}
        _event = <nil>
    }
    _loaded = True
    _version = <nil>
    _ns = object #2 (::omero::RString)
    {
        _val = omero.annotation.microsam.config
    }
    _name = <nil>
    _description = object #3 (::omero::RString)
    {
        _val = micro-SAM Configuration
    }
    _annotationLinksSeq = 
    {
    }
    _annotationLinksLoaded = True
    _annotationLinksCountPerOwner = {}
    _mapValue = 
    {
        [0] = <invalid value - expected ::omero::model::NamedValue>
        [1] = <invalid value - expected ::omero::model::NamedValue>
        [2] = <invalid value - expected ::omero::model::NamedValue>
        [3] = <invalid value - expected ::omero::model::NamedValue>
        [4] = <invalid value - expected ::omero::

❌ Error during annotation pipeline: invalid value for element 0 of `::omero::api::NamedValueList'


ValueError: invalid value for element 0 of `::omero::api::NamedValueList'

## 8. Display Results

Let's examine the tracking table and results.

In [ ]:
# Display the tracking table if available
if table_id is not None:
    try:
        tracking_df = ezomero.get_table(conn, table_id)
        print(f"📋 Tracking table contains {len(tracking_df)} rows")
        print(f"✅ Processed: {tracking_df['processed'].sum()} units")
        print(f"⏳ Pending: {(~tracking_df['processed']).sum()} units")
        
        # Show training/validation split
        if not config.training.segment_all:
            train_count = tracking_df['train'].sum()
            validate_count = tracking_df['validate'].sum() 
            print(f"🎓 Training samples: {train_count}")
            print(f"✅ Validation samples: {validate_count}")
        
        # Display the table
        print("\n📊 Tracking Table:")
        display(tracking_df)
        
    except Exception as e:
        print(f"❌ Error retrieving tracking table: {e}")
else:
    print("ℹ️ No tracking table was created")

## 9. Configuration Export/Import

Save your configuration for future use or sharing.

In [ ]:
# Save configuration to file
config_filename = f"annotation_config_{trainingset_name}.yaml"
config.save_yaml(config_filename)
print(f"💾 Configuration saved to: {config_filename}")

# Display the YAML configuration
print("\n📄 YAML Configuration:")
print("=" * 50)
print(config.to_yaml())

# Show how to load it back
print("\n🔄 To reuse this configuration:")
print(f"```python")
print(f"from omero_annotate_ai import load_config")
print(f"config = load_config('{config_filename}')")
print(f"```")

## 10. Advanced: Tool-Specific Parameters

The package supports different AI tools with tool-specific parameters.

In [ ]:
# Get micro-SAM specific parameters
microsam_params = config.get_microsam_params()
print(f"🔧 Micro-SAM parameters:")
for key, value in microsam_params.items():
    print(f"   {key}: {value}")

# Show configuration for different models
print("\n🤖 Available micro-SAM models:")
models = ["vit_b", "vit_l", "vit_h", "vit_b_lm"]
for model in models:
    print(f"   • {model}")

print(f"\n🔬 Current model: {config.microsam.model_type}")
print(f"🧠 3D processing: {config.microsam.three_d}")
print(f"⏰ Timepoint mode: {config.microsam.timepoint_mode}")
print(f"🔢 Z-slice mode: {config.microsam.z_slice_mode}")

## 11. Clean Up

In [ ]:
# Clean up temporary directory
try:
    shutil.rmtree(output_directory)
    print(f"🗑️ Removed temporary directory: {output_directory}")
except Exception as e:
    print(f"⚠️ Error removing temporary directory: {e}")

# Close OMERO connection
if 'conn' in locals() and conn is not None:
    conn.close()
    print("🔌 OMERO connection closed")

print("\n✨ Workflow completed successfully!")

## Summary

This notebook demonstrated the new `omero-annotate-ai` package features:

### ✅ Improvements over the original workflow:
- **Simplified imports**: Single package import instead of multiple module reloads
- **Interactive configuration**: User-friendly widget with validation
- **YAML configuration**: Save, load, and share configurations easily
- **Micro-SAM focus**: Specialized support for micro-SAM models
- **Clean API**: Pipeline-based approach with clear separation of concerns
- **Better error handling**: Comprehensive validation and error messages
- **Modular dependencies**: Install only what you need

### 🚀 Next Steps:
1. **Training**: Use the generated annotations to train custom models
2. **Inference**: Apply trained models to new datasets
3. **Batch processing**: Process multiple datasets with saved configurations
4. **Model comparison**: Compare results from different micro-SAM models

### 📚 Package Features:
- **Pip installable**: `pip install -e .`
- **Optional dependencies**: Install only needed components (`pip install -e .[omero]`)
- **Backward compatible**: Works with existing micro-SAM workflows
- **Extensible**: Easy to extend for additional functionality

### 🔬 Micro-SAM Models:
- **vit_b**: Fast, lightweight model
- **vit_l**: Balanced performance and speed  
- **vit_h**: High accuracy, slower
- **vit_b_lm**: Specialized model variant